<a href="https://colab.research.google.com/github/saurabh-parkar/Music-Genre-Classification/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


##  Setting up the Features Dataset

In [3]:
df = pd.read_csv('/content/drive/My Drive/fma_metadata/features.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,2

In [4]:
df

feature       chroma_cens  ...             zcr.5             zcr.6
0       statistics          kurtosis  ...              skew               std
1           number                01  ...                01                01
2         track_id               NaN  ...               NaN               NaN
3                2  7.1806526184e+00  ...  2.0898721218e+00  6.1448108405e-02
4                3  1.8889633417e+00  ...  1.7167237997e+00  6.9330163300e-02
...            ...               ...  ...               ...               ...
106572      155316         -0.490129  ...          0.955388         0.0123847
106573      155317         -0.461559  ...           1.28306         0.0190591
106574      155318          0.552473  ...          0.828569         0.0179043
106575      155319         -0.176901  ...           1.81874         0.0201334
106576      155320          0.489665  ...            4.6872          0.137205

[106577 rows x 519 columns]

In [0]:
cols = []
for i, j in zip(df, df.iloc[0]):
  cols.append(i+" "+j)

In [0]:
df.columns = cols

In [0]:
df = df.drop([0,1,2], axis = 0)

In [0]:
df = df.rename(columns = {'feature statistics': 'track_id'})

In [0]:
df2 = df

In [10]:
df = pd.read_csv('/content/drive/My Drive/fma_metadata/tracks.csv',skiprows=1)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df = df.drop(0)

In [0]:
df = df.rename(columns={"Unnamed: 0":"track_id"})

In [0]:
# Selecting only the part of dataset labelled as 'small'
df = df[df['subset']=='small']

In [14]:
df['genre_top'].unique()

array(['Hip-Hop', 'Pop', 'Folk', 'Experimental', 'Rock', 'International',
       'Electronic', 'Instrumental'], dtype=object)

In [15]:
# Checking for null values
df['genre_top'].isnull().any()

False

In [16]:
df[['track_id', 'genre_top']]

track_id genre_top
1             2   Hip-Hop
3             5   Hip-Hop
4            10       Pop
16          140      Folk
17          141      Folk
...         ...       ...
105714   154308   Hip-Hop
105715   154309   Hip-Hop
105816   154413       Pop
105817   154414       Pop
106386   155066   Hip-Hop

[8000 rows x 2 columns]

In [0]:
genres = list(df['genre_top'].unique())

In [20]:
genres

['Electronic',
 'Experimental',
 'Folk',
 'Hip-Hop',
 'Instrumental',
 'International',
 'Pop',
 'Rock']

In [0]:
genres.sort()

In [22]:
genres

['Electronic',
 'Experimental',
 'Folk',
 'Hip-Hop',
 'Instrumental',
 'International',
 'Pop',
 'Rock']

In [0]:
# Creating a mapping from genres to numeric labels
labels =  {genres[i]:i for i in range(len(genres))}

In [24]:
labels

{'Electronic': 0,
 'Experimental': 1,
 'Folk': 2,
 'Hip-Hop': 3,
 'Instrumental': 4,
 'International': 5,
 'Pop': 6,
 'Rock': 7}

In [0]:
# Creating a mapping from numeric labels to Genres
inv_labels = {i:genres[i] for i in range(len(genres))}

In [26]:
inv_labels

{0: 'Electronic',
 1: 'Experimental',
 2: 'Folk',
 3: 'Hip-Hop',
 4: 'Instrumental',
 5: 'International',
 6: 'Pop',
 7: 'Rock'}

In [0]:
tags = df[['track_id', 'genre_top']]

In [0]:
#File to Genre mapping
mapping = dict()
mapping = {str(i):j for i, j in zip(df['track_id'], df['genre_top'])}
  

In [29]:
mapping

{'2': 'Hip-Hop',
 '5': 'Hip-Hop',
 '10': 'Pop',
 '140': 'Folk',
 '141': 'Folk',
 '148': 'Experimental',
 '182': 'Rock',
 '190': 'Folk',
 '193': 'Folk',
 '194': 'Folk',
 '197': 'Folk',
 '200': 'Folk',
 '203': 'Folk',
 '204': 'Folk',
 '207': 'Folk',
 '210': 'Folk',
 '211': 'Folk',
 '212': 'Folk',
 '213': 'Pop',
 '255': 'Rock',
 '256': 'Rock',
 '368': 'Rock',
 '424': 'Experimental',
 '459': 'Rock',
 '534': 'Folk',
 '540': 'Folk',
 '546': 'Folk',
 '574': 'Rock',
 '602': 'Folk',
 '615': 'Experimental',
 '620': 'Folk',
 '621': 'Folk',
 '625': 'Folk',
 '666': 'International',
 '667': 'International',
 '676': 'Hip-Hop',
 '690': 'Rock',
 '694': 'Hip-Hop',
 '695': 'Hip-Hop',
 '704': 'International',
 '705': 'International',
 '706': 'International',
 '707': 'International',
 '708': 'International',
 '709': 'International',
 '714': 'Folk',
 '715': 'Folk',
 '716': 'Folk',
 '718': 'Folk',
 '777': 'Rock',
 '814': 'Folk',
 '821': 'Pop',
 '822': 'Pop',
 '825': 'Rock',
 '853': 'International',
 '890': '

In [0]:
# Selecting only the rows where the genre is one of the 8 genres
tarjeys = [labels[mapping[str(i)]] if str(i) in mapping.keys() else None for i in df2['track_id']]

In [31]:
tarjeys

[3,
 None,
 3,
 6,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 2,
 2,
 None,
 None,
 None,
 None,
 None,
 1,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 7,
 None,
 None,
 None,
 None,
 None,
 2,
 None,
 None,
 2,
 2,
 None,
 None,
 2,
 None,
 None,
 2,
 None,
 None,
 2,
 2,
 None,
 None,
 2,
 None,
 None,
 2,
 2,
 2,
 6,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 7,
 7,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [0]:
df2['targets'] = tarjeys

In [0]:
df2 = df2.dropna()

In [35]:
df2

track_id chroma_cens kurtosis  ...         zcr.6 std targets
3             2     7.1806526184e+00  ...  6.1448108405e-02     3.0
5             5     5.2756297588e-01  ...  4.4860601425e-02     3.0
6            10     3.7022454739e+00  ...  4.0800448507e-02     6.0
18          140     5.3357946873e-01  ...  5.7712450624e-02     2.0
19          141     1.7289805412e-01  ...  2.8665028512e-02     2.0
...         ...                  ...  ...               ...     ...
105716   154308            -0.677709  ...         0.0842977     3.0
105717   154309            -0.585059  ...           0.12948     3.0
105818   154413            -0.214509  ...          0.023821     6.0
105819   154414            -0.487371  ...         0.0457333     6.0
106388   155066            0.0442161  ...          0.030787     3.0

[8000 rows x 520 columns]

## Extracting a subset of columns that will be used for training

In [0]:
spike_cols = [col for col in df2.columns if 'spectral' in col]

In [37]:
spike_cols

['spectral_bandwidth kurtosis',
 'spectral_bandwidth.1 max',
 'spectral_bandwidth.2 mean',
 'spectral_bandwidth.3 median',
 'spectral_bandwidth.4 min',
 'spectral_bandwidth.5 skew',
 'spectral_bandwidth.6 std',
 'spectral_centroid kurtosis',
 'spectral_centroid.1 max',
 'spectral_centroid.2 mean',
 'spectral_centroid.3 median',
 'spectral_centroid.4 min',
 'spectral_centroid.5 skew',
 'spectral_centroid.6 std',
 'spectral_contrast kurtosis',
 'spectral_contrast.1 kurtosis',
 'spectral_contrast.2 kurtosis',
 'spectral_contrast.3 kurtosis',
 'spectral_contrast.4 kurtosis',
 'spectral_contrast.5 kurtosis',
 'spectral_contrast.6 kurtosis',
 'spectral_contrast.7 max',
 'spectral_contrast.8 max',
 'spectral_contrast.9 max',
 'spectral_contrast.10 max',
 'spectral_contrast.11 max',
 'spectral_contrast.12 max',
 'spectral_contrast.13 max',
 'spectral_contrast.14 mean',
 'spectral_contrast.15 mean',
 'spectral_contrast.16 mean',
 'spectral_contrast.17 mean',
 'spectral_contrast.18 mean',
 'spec

In [0]:

spike_cols = ['mfcc.40 mean',
 'mfcc.41 mean',
 'mfcc.42 mean',
 'mfcc.43 mean',
 'mfcc.44 mean',
 'mfcc.45 mean',
 'mfcc.46 mean',
 'mfcc.47 mean',
 'mfcc.48 mean',
 'mfcc.49 mean',
 'mfcc.50 mean',
 'mfcc.51 mean',
 'mfcc.52 mean',
 'mfcc.53 mean',
 'mfcc.54 mean',
 'mfcc.55 mean',
 'mfcc.56 mean',
 'mfcc.57 mean',
 'mfcc.58 mean',
 'mfcc.59 mean']

## Extracting advanced features from the mp3 files using LIBrosa

In [0]:
import librosa

y, sr = librosa.load('/content/drive/My Drive/genres1000/genres/rock/rock.00084.wav')
mfcc = librosa.feature.mfcc(
    y=y, sr=sr, hop_length=512, n_mfcc=13
)

In [39]:
mfcc.T[0:128, :].shape

(128, 13)

In [0]:
spectral_center = librosa.feature.spectral_centroid(
    y=y, sr=sr, hop_length=512
)

chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=512)

spectral_contrast = librosa.feature.spectral_contrast(
    y=y, sr=sr, hop_length=512
)

In [41]:
spectral_center

array([[4890.21369417, 3936.2978961 , 3011.49573721, ..., 2648.30938018,
        2491.52267597, 2152.44599698]])

### Extracting features for all the tracks

In [0]:
import os

data = np.zeros((1000, 128, 13), dtype=np.float64)
i = 0


basepath = "/content/drive/My Drive/genres1000/genres/"
tar = []
for entry in os.listdir(basepath):
  dir1 = os.path.join(basepath, entry) +'/'
  print(dir1)
  for filename in os.listdir(dir1):
   audio_path = filename
   y, sr = librosa.load(dir1 + audio_path)
   mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=512, n_mfcc=13)
   spectral_center = librosa.feature.spectral_centroid(y=y, sr=sr, hop_length=512)
   chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=512)
   spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=512)

   tar.append(filename.split('.')[0])
   print(filename)
   data[i, :, 0:13] = mfcc.T[0:128, :]
   #data[i, :, 13:14] = spectral_center.T[0:128, :]
   #data[i, :, 14:26] = chroma.T[0:128, :]
   #data[i, :, 26:33] = spectral_contrast.T[0:128, :]
   i+=1

/content/drive/My Drive/genres1000/genres/hiphop/
hiphop.00000.wav
hiphop.00001.wav
hiphop.00002.wav
hiphop.00003.wav
hiphop.00004.wav
hiphop.00005.wav
hiphop.00006.wav
hiphop.00007.wav
hiphop.00008.wav
hiphop.00009.wav
hiphop.00010.wav
hiphop.00011.wav
hiphop.00012.wav
hiphop.00013.wav
hiphop.00014.wav
hiphop.00015.wav
hiphop.00016.wav
hiphop.00017.wav
hiphop.00018.wav
hiphop.00019.wav
hiphop.00020.wav
hiphop.00021.wav
hiphop.00022.wav
hiphop.00023.wav
hiphop.00024.wav
hiphop.00025.wav
hiphop.00026.wav
hiphop.00027.wav
hiphop.00028.wav
hiphop.00029.wav
hiphop.00030.wav
hiphop.00031.wav
hiphop.00032.wav
hiphop.00033.wav
hiphop.00034.wav
hiphop.00035.wav
hiphop.00036.wav
hiphop.00037.wav
hiphop.00038.wav
hiphop.00039.wav
hiphop.00040.wav
hiphop.00041.wav
hiphop.00042.wav
hiphop.00043.wav
hiphop.00044.wav
hiphop.00045.wav
hiphop.00046.wav
hiphop.00047.wav
hiphop.00048.wav
hiphop.00049.wav
hiphop.00050.wav
hiphop.00051.wav
hiphop.00052.wav
hiphop.00053.wav
hiphop.00054.wav
hiphop.00055.wa

NoBackendError: ignored

In [0]:
data.shape

(1000, 128, 13)

In [0]:
full = []

for i,j in zip(data,tar):
  full.append([i,j])

In [0]:
import random

random.shuffle(full)

In [0]:
X = []
Y = []
for i in range(len(full)):
  X.append(full[i][0])
  Y.append(full[i][1])

In [0]:
genres = list(set(Y))

In [0]:
labels =  {genres[i]:i for i in range(len(genres))}
labels

{'blues': 8,
 'classical': 3,
 'country': 0,
 'disco': 5,
 'hiphop': 2,
 'jazz': 4,
 'metal': 9,
 'pop': 1,
 'reggae': 7,
 'rock': 6}

In [0]:
Y1 = [labels[i] for i in Y]

In [0]:
from keras.utils import np_utils
Y1 = np_utils.to_categorical(Y1, num_classes=10)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
trainX, testX, trainY, testY = train_test_split(X, Y1, test_size = 0.2)

In [0]:
trainX = np.array(trainX)

In [0]:
trainX.shape

(6000, 38)

In [0]:
testX = np.array(testX)

In [0]:
testX.shape

(200, 128, 13)

In [0]:
mfcc = df2[[ 'mfcc.40 mean',
 'mfcc.41 mean',
 'mfcc.42 mean',
 'mfcc.43 mean',
 'mfcc.44 mean',
 'mfcc.45 mean',
 'mfcc.46 mean',
 'mfcc.47 mean',
 'mfcc.48 mean',
 'mfcc.49 mean',
 'mfcc.50 mean',
 'mfcc.51 mean',
 'mfcc.52 mean',
 'mfcc.53 mean',
 'mfcc.54 mean',
 'mfcc.55 mean',
 'mfcc.56 mean',
 'mfcc.57 mean',
 'mfcc.58 mean',
 'mfcc.59 mean']]

In [0]:
spectral_center = df2[['spectral_centroid.2 mean']]

In [0]:
chroma = df2[[ 'chroma_stft.24 mean',
 'chroma_stft.25 mean',
 'chroma_stft.26 mean',
 'chroma_stft.27 mean',
 'chroma_stft.28 mean',
 'chroma_stft.29 mean',
 'chroma_stft.30 mean',
 'chroma_stft.31 mean',
 'chroma_stft.32 mean',
 'chroma_stft.33 mean',
 'chroma_stft.34 mean',
 'chroma_stft.35 mean']]

In [0]:
spectral_contrast = df2[['spectral_contrast.14 mean',
 'spectral_contrast.15 mean',
 'spectral_contrast.16 mean',
 'spectral_contrast.17 mean',
 'spectral_contrast.18 mean',
 'spectral_contrast.19 mean',
 'spectral_contrast.20 mean']]

## Final subset of features that will be used for training (From the original CSV file)

In [0]:
df3 = df2[[ 'mfcc.40 mean',
 'mfcc.41 mean',
 'mfcc.42 mean',
 'mfcc.43 mean',
 'mfcc.44 mean',
 'mfcc.45 mean',
 'mfcc.46 mean',
 'mfcc.47 mean',
 'mfcc.48 mean',
 'mfcc.49 mean',
 'mfcc.50 mean',
 'mfcc.51 mean',
 'mfcc.52 mean',
 'mfcc.53 mean',
 'mfcc.54 mean',
 'mfcc.55 mean',
 'mfcc.56 mean',
 'mfcc.57 mean',
 'mfcc.58 mean',
 'mfcc.59 mean',
 'chroma_stft.24 mean',
 'chroma_stft.25 mean',
 'chroma_stft.26 mean',
 'chroma_stft.27 mean',
 'chroma_stft.28 mean',
 'chroma_stft.29 mean',
 'chroma_stft.30 mean',
 'chroma_stft.31 mean',
 'chroma_stft.32 mean',
 'chroma_stft.33 mean',
 'chroma_stft.34 mean',
 'chroma_stft.35 mean', 'spectral_contrast.14 mean',
 'spectral_contrast.15 mean',
 'spectral_contrast.16 mean',
 'spectral_contrast.17 mean',
 'spectral_contrast.18 mean',
 'spectral_contrast.19 mean',
 'spectral_contrast.20 mean','targets',]]

In [0]:
X = df3.iloc[:,1:-1]

In [0]:
Y = df3['targets']

In [45]:
X

mfcc.41 mean  ... spectral_contrast.20 mean
3       1.1669667816e+02  ...          3.8268646240e+01
5       1.3221507263e+02  ...          3.6413608551e+01
6       1.5704008484e+02  ...          3.4886196136e+01
18      1.3933282471e+02  ...          3.6932270050e+01
19      1.5571632385e+02  ...          2.9015615463e+01
...                  ...  ...                       ...
105716           152.342  ...                   23.4276
105717           104.314  ...                   27.3598
105818           155.606  ...                   32.4246
105819            156.15  ...                   32.9808
106388           160.116  ...                   18.4114

[8000 rows x 38 columns]

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from keras.utils import np_utils
Y = np_utils.to_categorical(Y, num_classes=8)

In [0]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.25)

In [57]:
trainX.shape

(6000, 38)

In [0]:
trainX = np.array(trainX)

In [0]:
testX = np.array(testX)

In [0]:
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [0]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Activation, Flatten, Conv2D,MaxPooling2D, AveragePooling2D, Conv1D, MaxPooling1D, TimeDistributed,LSTM, BatchNormalization, CuDNNLSTM, LSTM, Conv3D


#### Note: 2 different types of inputs can be used as training data - 1) From the actual features extracted using LIBrosa, 2) Using the subset of the  original CSV file.

## Building a simple neural network for genre classification

In [62]:
'''
model = Sequential()


model.add(LSTM(128, dropout=0.05, recurrent_dropout=0.35, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(32,  dropout=0.05, recurrent_dropout=0.35, return_sequences=False))

model.add(Dense(10, activation='softmax'))

'''
model = Sequential()
model.add(LSTM(128, input_shape=(1,38), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(8, activation='softmax'))



model.compile(loss="categorical_crossentropy", optimizer= 'adam', metrics=["accuracy"])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
model.summary()

In [0]:

model.fit(
    trainX,
    trainY,
    batch_size=64,
    epochs=100,
    validation_split=0.1
)

In [0]:
model.evaluate(X_test,testY)